In [66]:
import os
import re
import json
import random
import sys
import asyncio
import pickle
import datetime
import gurobipy as gp
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel
from colorama import Fore, Style

In [6]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [7]:
DATA_DIR = '../data'
DATASET_NAME = 'NL4OPT' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'nl4opt.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in demo_samples]
answers = [a for _, a in demo_samples]

2024-09-17 15:08:04.869 | DEBUG    | utils:read_txt_file:14 - Reading file: ../data\NL4OPT\nl4opt.txt
2024-09-17 15:08:04.870 | DEBUG    | utils:read_txt_file:16 - File read successfully: ../data\NL4OPT\nl4opt.txt
2024-09-17 15:08:04.871 | INFO     | utils:get_nl4opt_qas:35 - Number of questions: 245
2024-09-17 15:08:04.871 | INFO     | utils:get_nl4opt_qas:36 - Number of answers: 245
2024-09-17 15:08:04.872 | INFO     | utils:get_demo_and_test_samples:47 - Number of demo samples: 20
2024-09-17 15:08:04.872 | INFO     | utils:get_demo_and_test_samples:48 - Number of test samples: 225


In [8]:
class Code(BaseModel):
    reasoning: List[str]
    code: str     

In [15]:
sys_prompt = """You are an expert in optimization problems and domain specific language generation. Your task is to convert the textual optimization text into a piece of code.
Here are some examples that you should refer to:\n"""

example = """
QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 13 * y <= 1345)
m.addConstr(43 * x + 4 * y <= 346)
m.addConstr(56 * x + 45 * y <= 1643)

QUESTION:
Ben is growing apples and pears on his orchard. He has 50 acres available on which he must grow a minimum of 5 acres of apples and a minimum of 10 acres of pears to meet demands. The profit per apple is $2 and the profit per pear is $4. He prefers to grow more pears than apples but limitations in his workforce allow him to grow at most twice the amount of pears as apples. How many of each fruit should Ben grow in order to maximize his profit? What is that profit?
CODE:
x = m.addVar(name="apples", vtype=gp.GRB.INTEGER)
y = m.addVar(name="pears", vtype=gp.GRB.INTEGER)
m.setObjective(2 * x + 4 * y, gp.GRB.MAXIMIZE)
m.addConstr(x + y <= 50)
m.addConstr(x >= 5)
m.addConstr(y >= 10)
m.addConstr(y <= 2 * x)
"""

sys_prompt = sys_prompt + example + "\nPlease finish the task think step by step."
print(sys_prompt)

You are an expert in optimization problems and domain specific language generation. Your task is to convert the textual optimization text into a piece of code.
Here are some examples that you should refer to:

QUESTION:
A car manufacturer makes two types of car oils: Oil Max and Oil Max Pro. A container of Oil Max contains 46 grams of substance A, 43 grams of substance B and 56 grams of substance C. A container of Oil Max Pro contains 13 grams of substance A, 4 grams of substance B and 45 grams of substance C. The car manufacturer has 1345 grams of substance A, 346 grams of substance B, 1643 grams of substance C. In addition, the profit per container of Oil Max is $10 and the profit per container of Oil Max Pro is $15. How many containers of each of oil should the car manufacturer make to maximize profit?
CODE:
x = m.addVar(name="Oil Max", vtype=gp.GRB.INTEGER)
y = m.addVar(name="Oil Max Pro", vtype=gp.GRB.INTEGER)
m.setObjective(10 * x + 15 * y, gp.GRB.MAXIMIZE)
m.addConstr(46 * x + 1

In [16]:
batch_size = 8
lp_reasoning_list = []
for idx in tqdm(range(0, len(questions), batch_size)):
    batch = questions[idx:idx+batch_size]
    
    tasks = [asyncclient.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0,
        response_format=Code,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": f"QUESTION: {q}"}
        ]) for q in batch
    ]

    combined_responses = await asyncio.gather(*tasks)
    lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

100%|██████████| 3/3 [00:21<00:00,  7.07s/it]


In [17]:
filename = 'e2e_codegen_small_gurobi_' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(lp_reasoning_list, f)

In [18]:
codes = [lp_reasoning_list[i].code for i in range(len(lp_reasoning_list))]

In [69]:
prefix = """
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = gp.Model(env=env)
"""
                
suffix = """
m.optimize()
"""

def complement_code(code: str) -> float:
    return prefix + code + suffix

In [70]:
def clean_code(code: str) -> str:
    cleand_code = []
    for line in code.split('\n'):
        line = line.strip()
        if line.startswith('m.addConstr') and not re.findall(r'<=|>=', line):
            line = re.sub(r'<', r'<=', line)
            line = re.sub(r'>', r'>=', line)
        cleand_code.append(line)
    return '\n'.join(cleand_code)

In [71]:
def execute_code(code: str) -> float:
    ex_locals = {}
    exec(code, None, ex_locals)
    
    try:
        return ex_locals["m"].objVal
    except Exception as e:
        # print(e)
        return np.inf

In [72]:
pred_answers = []
for i, code_str in enumerate(codes):
    try:
        cleaned_code = clean_code(code_str)
        code = complement_code(cleaned_code)
        ans = execute_code(code)
        loguru.logger.info(f"question {i} obtain answer")
        pred_answers.append(ans)
    except Exception as e:
        loguru.logger.error(f"Error for question {i}: {e}")
        pred_answers.append("Error")

2024-09-17 15:47:34.965 | INFO     | __main__:<module>:7 - question 0 obtain answer
2024-09-17 15:47:34.968 | INFO     | __main__:<module>:7 - question 1 obtain answer
2024-09-17 15:47:34.971 | INFO     | __main__:<module>:7 - question 2 obtain answer
2024-09-17 15:47:34.973 | INFO     | __main__:<module>:7 - question 3 obtain answer
2024-09-17 15:47:34.975 | INFO     | __main__:<module>:7 - question 4 obtain answer
2024-09-17 15:47:34.976 | INFO     | __main__:<module>:7 - question 5 obtain answer
2024-09-17 15:47:34.978 | INFO     | __main__:<module>:7 - question 6 obtain answer
2024-09-17 15:47:34.980 | INFO     | __main__:<module>:7 - question 7 obtain answer
2024-09-17 15:47:34.981 | INFO     | __main__:<module>:7 - question 8 obtain answer


2024-09-17 15:47:34.983 | INFO     | __main__:<module>:7 - question 9 obtain answer
2024-09-17 15:47:34.984 | INFO     | __main__:<module>:7 - question 10 obtain answer
2024-09-17 15:47:34.985 | INFO     | __main__:<module>:7 - question 11 obtain answer
2024-09-17 15:47:34.987 | INFO     | __main__:<module>:7 - question 12 obtain answer
2024-09-17 15:47:34.988 | INFO     | __main__:<module>:7 - question 13 obtain answer
2024-09-17 15:47:34.989 | INFO     | __main__:<module>:7 - question 14 obtain answer
2024-09-17 15:47:34.991 | INFO     | __main__:<module>:7 - question 15 obtain answer
2024-09-17 15:47:34.993 | INFO     | __main__:<module>:7 - question 16 obtain answer
2024-09-17 15:47:34.995 | INFO     | __main__:<module>:7 - question 17 obtain answer
2024-09-17 15:47:34.997 | INFO     | __main__:<module>:7 - question 18 obtain answer
2024-09-17 15:47:34.999 | INFO     | __main__:<module>:7 - question 19 obtain answer


In [73]:
print([round(pa, 4) for pa in pred_answers])

[540.0, 166.0, 950.0, 37083.3333, 342750.0, 7000.0, 100.0, 11980.0, 480.0, 142.0, 465.0, inf, 67.0, inf, 1500.0, 511.4286, 1060.0, 11275.0, 20.0, 50.0]


In [74]:
print(answers)

['540.0', '166.66666666666669', '950.0', '36900.0', '342750.0', '7000.0', '100.0', '11980.0', '480.0', '142.0', '465.0', inf, '67.0', inf, '1500.0', '511.42857142857133', '1060.0', '2500.0', '20.0', '-99999']


In [75]:
correct = []
for p, r in zip(pred_answers, answers):
    if p == 'Error':
        continue
    if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < 1e-2):
        correct.append(True)
    else:
        correct.append(False)

In [76]:
(sum(correct) / len(answers)) * 100

95.0